In this Notebook, we will train a DRL Agent using an multi-period EIIE model as a policy function for the Dow Jones Industrial Average. The evaluation and performance including the agent's actions are displayed more explicity in the notebook.

## 1. Importing Libraries

In [1]:
import yfinance as yf
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from modules.scalers import GroupByScaler
from sklearn.preprocessing import MaxAbsScaler
import json
import torch

from modules.environment import MultiPeriodPortfolioOptimizationEnv
from modules.architectures import MultiPeriodEIIE
from modules.models import DRLAgent

sns.set()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## 2. Getting Config Data

In [2]:
config_file_name = "config.json"
with open(config_file_name, "r") as jsonfile:
    config_data = json.load(jsonfile)

In [3]:
djia = config_data["tickers"]["America"]["DJIA"]
NUM_ASSETS = len(djia)
len(djia)

30

In [4]:
start_date = config_data["timeframe"]["djia"]["start_date"]
end_date = config_data["timeframe"]["djia"]["end_date"]
data_interval = config_data["data_interval"]
test_ratio = config_data["train_test_ratio"]
random_seed = config_data["random_state_seed"]

## 3. Data Retrieval

In [5]:
portfolio_raw_df = yf.download(tickers=djia, start=start_date, end=end_date, interval=data_interval)

portfolio_raw_df.fillna(method="bfill", inplace=True)
portfolio_raw_df = portfolio_raw_df.stack(level=1).rename_axis(["Date", "Ticker"]).reset_index(level=1)
portfolio_raw_df = portfolio_raw_df.drop("Adj Close", axis=1)
portfolio_raw_df.columns.name = None
portfolio_raw_df = portfolio_raw_df.reset_index()
portfolio_raw_df.Date = portfolio_raw_df.Date.astype(str)
portfolio_raw_df.columns = ["date", "tic", "close", "high", "low", "open", "volume"]
portfolio_raw_df = portfolio_raw_df[["date", "tic", "close", "high", "low", "volume"]]

df_portfolio_raw_train, df_portfolio_raw_test = train_test_split(portfolio_raw_df, test_size=test_ratio, shuffle=False, random_state=random_seed)
df_portfolio_train = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_raw_train)
df_portfolio_test = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_raw_test)


[*********************100%%**********************]  30 of 30 completed
/var/folders/02/rmgfx9wj0s99nn4tzfthtrnc0000gn/T/ipykernel_33774/2286451695.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  portfolio_raw_df.fillna(method="bfill", inplace=True)
/var/folders/02/rmgfx9wj0s99nn4tzfthtrnc0000gn/T/ipykernel_33774/2286451695.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  portfolio_raw_df = portfolio_raw_df.stack(level=1).rename_axis(["Date", "Ticker"]).reset_index(level=1)


## 4. DRL Environment
### 4.1 Hyperparameter selection

In [6]:
# SETTING HYPERPARAMETERS
FEATURES = ["close", "high", "low", "volume"]
N = config_data["lookback_window"]
T = config_data["multi_step_horizon"]
NUM_FEATURES = len(FEATURES)
experiment_type = "EIIE_DJIA" 
N, T

(24, 3)

### 4.2 Train Environment Initialization

In [7]:
train_environment = MultiPeriodPortfolioOptimizationEnv(
    df_portfolio_train,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=N,
    multi_period_horizon=T,
    features=FEATURES,
    normalize_df=None,
    is_train_mode=True,
    experiment_type=experiment_type
)

### 4.3 Model Parameters

In [8]:
# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.0001,
    "policy": MultiPeriodEIIE,
    "multi_period_horizon": T
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "initial_features": NUM_FEATURES,
    "k_size": 5, # Size of Initial Kernel
    "time_window": N,
    "prediction_horizon": T
}

### 4.4 Agent initialization

In [9]:
model = DRLAgent(train_environment).get_model("pg", device, model_kwargs, policy_kwargs)

## 5. Model Training

In [10]:
print("TRAINING AGENT.....")
DRLAgent.train_model(model, episodes=35)

TRAINING AGENT.....


  0%|          | 0/35 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 2051062.8386826129
Final accumulative portfolio value: 20.510628386826127
Maximum DrawDown: -0.7375416177654324
Sharpe ratio: 1.3629840311500507


  3%|▎         | 1/35 [00:01<00:53,  1.57s/it]

Initial portfolio value:100000
Final portfolio value: 2051075.9844941504
Final accumulative portfolio value: 20.510759844941504
Maximum DrawDown: -0.7375417217224748
Sharpe ratio: 1.3629856533296734


...

 94%|█████████▍| 33/35 [00:48<00:02,  1.48s/it]

Initial portfolio value:100000
Final portfolio value: 2052130.372921066
Final accumulative portfolio value: 20.521303729210658
Maximum DrawDown: -0.7375298285335696
Sharpe ratio: 1.3631594124554802


 97%|█████████▋| 34/35 [00:50<00:01,  1.47s/it]

Initial portfolio value:100000
Final portfolio value: 2052188.1962844545
Final accumulative portfolio value: 20.521881962844546
Maximum DrawDown: -0.737528800789583
Sharpe ratio: 1.363169457037799


100%|██████████| 35/35 [00:51<00:00,  1.47s/it]


In [11]:
print("PERSISTING MODEL.....")
torch.save(model.train_policy.state_dict(), f"models/policy_{experiment_type}.pt")

PERSISTING MODEL.....


In [12]:
print("TESTING.....")
MEIIE_results = {
    "training": train_environment._asset_memory["final"],
    "test": {}
}

TESTING.....


## 6. Model Evaluation

In [13]:
test_environment = MultiPeriodPortfolioOptimizationEnv(
    df_portfolio_test,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=N,
    multi_period_horizon=T,
    features=FEATURES,
    normalize_df=None,
    is_train_mode=False,
    experiment_type=experiment_type
)

In [14]:
policy = MultiPeriodEIIE(
    initial_features=NUM_FEATURES,
    k_size=5,
    time_window=N,
    prediction_horizon=T
)

In [15]:
policy.load_state_dict(torch.load(f"models/policy_{experiment_type}.pt"))

<All keys matched successfully>

In [16]:
DRLAgent.DRL_validation(model=model, test_env=test_environment, policy=policy)

Initial portfolio value:100000
Final portfolio value: 386176.7602853223
Final accumulative portfolio value: 3.8617676028532233
Maximum DrawDown: -0.3424335191363347
Sharpe ratio: 1.127687493513427


## 7. Postprocessing

In [17]:
MEIIE_results["test"]["value"] = test_environment._asset_memory["final"]

In [18]:
drl_portfolio_performance = test_environment._asset_memory["final"][1:]
len(drl_portfolio_performance)

70

In [19]:
drl_portfolio_performance = [(x/100000) for x in drl_portfolio_performance]

In [20]:
performance_dataset = pd.read_csv("data/processed/performances_djia.csv", index_col=0)

In [21]:
performance_dataset["DRL_EIIE"] = drl_portfolio_performance

In [22]:
performance_dataset.to_csv("data/processed/performances_djia.csv")